<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/main/Copy_of_NLP_ass2_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#loading data from github repo

import pandas as pd
url_diagn_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
url_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-test.csv'
url_train ='https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-train.csv'

diagnostic_test = pd.read_csv(url_test)
train = pd.read_csv(url_train)
test = pd.read_csv(url_test)


,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1
...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,1
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,0
857,67018,3 people just unfollowed me for talking about ...,1
858,50665,#WednesdayWisdom Antifa calls the right fascis...,0
